<a href="https://colab.research.google.com/github/laralex/Sk-DL2021-FinalProject/blob/main/notebooks/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Signal recovery from nonlinear distortion in optical communications
Deep Learning 2021 course at Skoltech, Russia.

A quickstart notebook to reproduce the experiments. Suggested to use on Google Colab.

<h3><span style="color:blue;">Configure only there</span></h3>

In [ ]:
MAIN_DIR_NAME = 'SignalRecoveryExperiments'
CONFIG_PATH = 'configs/mnist.yaml'

ON_GOOGLE_COLAB = True
MOUNT_GOOGLE_DRIVE = True # can't mount on Windows
GPUS_INDICES = [0] # None for CPU, list for multiple GPU

GIT_BRANCH = 'main'
GIT_PULL = True

### Destination of the experimental results (logs, checkpoints)

In [ ]:
current_dir = '/content' if ON_GOOGLE_COLAB else '.'
if MOUNT_GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount(f'{current_dir}/drive')
    root_dir = f'{current_dir}/drive/MyDrive/{MAIN_DIR_NAME}'
else:
    root_dir = f'{current_dir}/{MAIN_DIR_NAME}'

### Our repository and prerequisites

In [ ]:
!git clone https://github.com/laralex/Sk-DL2021-FinalProject $root_dir
%cd $root_dir
!git checkout $GIT_BRANCH
!pip install pytorch_lightning
!pip install jsonargparse

import os
import sys
sys.path.append(root_dir)

### TensorBoard logs

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $root_dir/logs

### Launch training

In [ ]:
# useful for contributors
if GIT_PULL:
    !git pull origin $GIT_BRANCH

In [ ]:
%load_ext autoreload
%autoreload 1

%aimport train

In [ ]:
train.main(
    root_dir = root_dir,
    config_path = f"{root_dir}/{CONFIG_PATH}", 
    gpu_indices = GPUS_INDICES)